In [2]:
%cd  G:\Shared drives\INVent\mfk58\Dataset\

#ensure file directory is correct

%ls 

G:\Shared drives\INVent\mfk58\Dataset
 Volume in drive G is Google Drive
 Volume Serial Number is 1983-1116

 Directory of G:\Shared drives\INVent\mfk58\Dataset

05/30/2023  03:50 PM    <DIR>          .
06/07/2023  12:34 PM    <DIR>          ..
05/30/2023  03:50 PM    <DIR>          CCF_Pre_Resampled
05/30/2023  03:53 PM    <DIR>          UH_Pre_Resampled
               0 File(s)              0 bytes
               4 Dir(s)  256,880,898,048 bytes free


In [3]:
%pip install Pillow
%pip install numpy


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [9]:
import glob
import SimpleITK as sitk

#check some basic sanity check information to see if the masks are the same
def hasMask(name):
    files = glob.glob(name+"/*.mha")

    reader = sitk.ImageFileReader()

    if(len(files)<2):
        return False

    reader.SetFileName(files[0])
    reader.LoadPrivateTagsOn()
    reader.ReadImageInformation()

    size1 = reader.GetSize()  
    res1 = reader.GetSpacing() 

    
    reader.SetFileName(files[1])
    reader.LoadPrivateTagsOn()
    reader.ReadImageInformation()

    size2 = reader.GetSize()  
    res2 = reader.GetSpacing()
    
    if not (size1 == size2 and res2 == res2):
        return False


    return True


def axial(name):
    files = glob.glob(name+"/*.mha")
    if("ax" in files[0] and "ax" in files[1]):
        return True
    return False


import pandas as pd
newSplits = pd.read_excel(r'G:\Shared drives\INVent\mfk58\New-Dataset-split.xlsx',sheet_name='splits').to_numpy()


CCF_count = 0

CCF_missing = [] #missing masks
CCF_list = []

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\*'):
    files = glob.glob(name+'/*.mha')
    if not (hasMask(name)):
        CCF_missing.append(name)
        continue
    if not ((files[0].replace(name+"\\","").replace(".mha","") in newSplits) or (files[1].replace(name+"\\","").replace(".mha","") in newSplits)):
        continue
    if axial(name):
        CCF_count +=1
        CCF_list.append(name)
    

    

print("Total CCF: " + str(CCF_count))
print("Missing masks for CCF:" + str(CCF_missing))




UH_count = 0

UH_missing = [] #missing masks
UH_list = []

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\\*'):
    files = glob.glob(name+'/*.mha')

    if not (hasMask(name)):
        UH_missing.append(name)
        continue
    if not ((files[0].replace(name+"\\","").replace(".mha","") in newSplits) or (files[1].replace(name+"\\","").replace(".mha","") in newSplits)):
        continue
    if axial(name):
        UH_count +=1
        UH_list.append(name)

    

print("Total UH: " + str(UH_count))
print("Missing masks for UH:" + str(UH_missing))

print("Total: " +str(UH_count+CCF_count))
print("Expected: 139")


Total CCF: 40
Missing masks for CCF:[]
Total UH: 99
Missing masks for UH:[]
Total: 139
Expected: 139


Calculating the greater statistics

In [5]:
%pip install pandas
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [11]:
import pandas as pd
import numpy as np

split = pd.read_excel(r'G:\Shared drives\INVent\mfk58\New-Dataset-split.xlsx',sheet_name="splits")
split = split.to_numpy();
#print(split[[1,2]])



names = UH_list
adjusted = []
for end in names:
    end = "UH_" + end.rsplit("\\",1)[-1] + "_pre_ax_raw_resampled"
    adjusted.append(end)

names = CCF_list
for end in names:
    end = "CCF_" + end.rsplit("\\",1)[-1] + "_pre_ax_raw_resampled"
    adjusted.append(end)


#these files do not match the general naming scheme, but they are present
adjusted = list(map(lambda x: x.replace("CCF_RectalCA_011_pre_ax_raw_resampled", "CCF_RectalCA_011_pre_ax_resampled"), adjusted))
adjusted = list(map(lambda x: x.replace("CCF_RectalCA_060_pre_ax_raw_resampled", "CCF_RectalCA_060_pre_ax_resampled"), adjusted))

print("Files that are not in the splitting list")
for name in adjusted:
    if not(np.isin(name,split)):
        print(name)


Files that are not in the splitting list


In [14]:

def removeNaN(x):
    temp = x[~pd.isnull(x)]
    return temp

x = removeNaN(split[:,0:3].flatten())
#x = np.append(x, split[0:10,3])
x.sort()
#print(x)

count = 0
for name in x:

    if name.split('_',1)[0] == "UH":
        break
    count += 1
    
print("Training set CCF: " + str(count))
print("Training set UH: " + str(len(x)-count))

print("Ext val set CCF: " + str(CCF_count-count))
print("Ext val set UH: " + str(UH_count-len(x)+count))




Training set CCF: 32
Training set UH: 97
Ext val set CCF: 8
Ext val set UH: 2


In [8]:


"""
###THIS IS A REALLY TERRIBLE IMPLEMENTATION - extremely inefficient
comList = CCF_list + UH_list
#print(comList)


maxWidth = 0
madDepth = 0
maxHeight = 0
maxVol = 0

for name in comList:
    files = glob.glob(name+"/*")
    f1 = sitk.ReadImage(files[0], imageIO="MetaImageIO")
    f2 = sitk.ReadImage(files[1], imageIO="MetaImageIO")

    w1 = f1.GetWidth()
    w2 = f2.GetWidth()

    h1 = f1.getHeight()
    h2 = f2.GetHeight()

    d1 = f1.GetDepth()
    d2 = f2.GetDepth()


    print("running " +name)
    


    #del f1, f2, files #needed this to not make the memory overflow

"""


'\n###THIS IS A REALLY TERRIBLE IMPLEMENTATION - extremely inefficient\ncomList = CCF_list + UH_list\n#print(comList)\n\n\nmaxWidth = 0\nmadDepth = 0\nmaxHeight = 0\nmaxVol = 0\n\nfor name in comList:\n    files = glob.glob(name+"/*")\n    f1 = sitk.ReadImage(files[0], imageIO="MetaImageIO")\n    f2 = sitk.ReadImage(files[1], imageIO="MetaImageIO")\n\n    w1 = f1.GetWidth()\n    w2 = f2.GetWidth()\n\n    h1 = f1.getHeight()\n    h2 = f2.GetHeight()\n\n    d1 = f1.GetDepth()\n    d2 = f2.GetDepth()\n\n\n    print("running " +name)\n    \n\n\n    #del f1, f2, files #needed this to not make the memory overflow\n\n'

In [15]:
import pathlib
import SimpleITK as sitk
import numpy as np

In [16]:


#find maximum values


comList = CCF_list + UH_list
#print(comList)

dimensions = [0,0,0,0,0] #x,y,z, x-y area, vol
dfiles = ["","","","",""]
max = [0,0,0,0,0]


reader = sitk.ImageFileReader()




print (len(comList))
for name in comList:
    #print("running " +name)
    files = glob.glob(name+"/*.mha")
    if not ("ax" in files[0] and "ax" in files[1]):
        continue
    for file in files:
        reader.SetFileName(file)
        reader.LoadPrivateTagsOn()
        reader.ReadImageInformation()

        size = reader.GetSize()  
        xres, yres, zres = reader.GetSpacing() 
        size = list(size)
        size.append(np.prod(size[0:2]))
        size.append(np.prod(size[0:3]))

        for dim in range(len(dimensions)):
            if size[dim] > dimensions[dim]:
                dimensions[dim] = size[dim]
                dfiles[dim] = name
                max[dim] = size

            if not (xres == 1 and yres == 1 and zres == 1):
                print("resolution is incorrect for: " + name)


table = [dimensions,dfiles,max]
print(table)
        






    
    



139
[[320, 300, 286, 90000, 19440000], ['G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_027', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_011', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_089', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_011', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_031'], [[320, 210, 99, 67200, 6652800], [300, 300, 144, 90000, 12960000], [200, 200, 286, 40000, 11440000], [300, 300, 144, 90000, 12960000], [300, 300, 216, 90000, 19440000]]]


In [17]:


#find minimum values



#based on the maximums obtained from the previous block - these should be large enough
dimensions = [10000,10000,10000,1000000,31276000] #x,y,z, x-y area, vol
dfiles = ["","","","",""]
max = [0,0,0,0,0]


reader = sitk.ImageFileReader()




print (len(comList))
for name in comList:
    #print("running " +name)
    files = glob.glob(name+"/*.mha")
    if not ("ax" in files[0] and "ax" in files[1]):
        continue
    for file in files:
        reader.SetFileName(file)
        reader.LoadPrivateTagsOn()
        reader.ReadImageInformation()

        size = reader.GetSize()  
        xres, yres, zres = reader.GetSpacing() 
        size = list(size)
        size.append(np.prod(size[0:2]))
        size.append(np.prod(size[0:3]))

        for dim in range(len(dimensions)):
            if size[dim] < dimensions[dim]:
                dimensions[dim] = size[dim]
                dfiles[dim] = name
                max[dim] = size

            if not (xres == 1 and yres == 1 and zres == 1):
                print("resolution is incorrect for: " + name)


mintable = [dimensions,dfiles,max]
print(mintable)
        






    
    



139
[[150, 135, 70, 22500, 1920000], ['G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_039', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_124', 'G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\\RectalCA_158', 'G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\\RectalCA_039', 'G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\\RectalCA_038'], [[150, 150, 200, 22500, 4500000], [180, 135, 173, 24300, 4203900], [180, 180, 70, 32400, 2268000], [150, 150, 200, 22500, 4500000], [160, 160, 75, 25600, 1920000]]]


### Statistics on tumor size

Need some greater information on tumor size:
-Need the maximum size necessary in each dimension- i.e., the maximum sizes of the tumor

In [18]:
#testing this implementation to sort out the locations of certain values and only that value

import numpy as np

a = np.array([[[36,  1, 72],
         [76, 50, 23],
         [28, 68, 17],
         [84, 75, 69]],
 
        [[ 5, 15, 93],
         [92, 92, 88],
         [11, 54, 21],
         [87, 76, 81]]])

vals = np.array([1])
inds = a == vals[:, None, None, None]
a[~np.any(inds, axis = 0)] = 0
a

array([[[0, 1, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]])

In [31]:
def filter(arr,val):
    a = arr
    vals = np.array([val])
    inds = a == vals[:, None, None, None]
    a[~np.any(inds, axis = 0)] = 0
    
    return a



def crop_image(img):

    r = np.any(img, axis=(1, 2))
    c = np.any(img, axis=(0, 2))
    z = np.any(img, axis=(0, 1))

    rmin, rmax = np.where(r)[0][[0, -1]]
    cmin, cmax = np.where(c)[0][[0, -1]]
    zmin, zmax = np.where(z)[0][[0, -1]]
    return img[rmin:rmax,cmin:cmax,zmin:zmax]


comList = CCF_list + UH_list


def get_bounding_box(x):
    """ Calculates the bounding box of a ndarray"""
    mask = x == 0
    bbox = []
    all_axis = np.arange(x.ndim)
    for kdim in all_axis:
        nk_dim = np.delete(all_axis, kdim)
        mask_i = mask.all(axis=tuple(nk_dim))
        dmask_i = np.diff(mask_i)
        idx_i = np.nonzero(dmask_i)[0]

        low = min( idx_i )
        high = max( idx_i )
        s = slice(low + 1, high + 1)
        bbox.append(s)
    return bbox


def filterA(a,val):
    arr = a
    arr[arr != val] = 0
    return arr

In [38]:
dimensions = [0,0,0,0,0] #x,y,z, x-y area, vol
dfiles = ["","","","",""]
maxs = [0,0,0,0,0]
import pandas as pd

excl = [r"G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\RectalCA_132\UH_RectalCA_132_pre_ax_raw_label_resampled.mha",r"G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\RectalCA_172\UH_RectalCA_172_pre_ax_raw_label_resampled.mha"]


print (len(comList))
for name in comList:
    #print("running " +name)
    files = glob.glob(name+"/*.mha")
    if not ("ax" in files[0] and "ax" in files[1]):
        continue
    for file in files:
        if not ("label" in file):
            continue
        
        if file in excl:
            continue

        image = sitk.ReadImage(file, imageIO="MetaImageIO")
        arr = sitk.GetArrayFromImage(image)
        arr = filterA(arr,1)
        for i in range(2,10):
            if i in arr:
                print("not good")
                break
        print(file)
        bbx = get_bounding_box(arr)
        print(bbx)
        
        arr = arr[bbx[0].start:bbx[0].stop,bbx[1].start:bbx[1].stop,bbx[2].start:bbx[2].stop]
        size = list(arr.shape)

        size.append(np.prod(size[0:2]))
        size.append(np.prod(size[0:3]))

        for dim in range(len(dimensions)):
            if size[dim] > dimensions[dim]:
                dimensions[dim] = size[dim]
                dfiles[dim] = name
                maxs[dim] = size



table = [dimensions,dfiles,maxs]
print(table)
        


139
G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_001\CCF_RectalCA_001_pre_ax_label_raw_resampled.mha
[slice(136, 168, None), slice(108, 138, None), slice(84, 116, None)]
G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_014\CCF_RectalCA_014_pre_ax_label_raw_resampled.mha
[slice(75, 123, None), slice(134, 171, None), slice(85, 124, None)]
G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_011\CCF_RectalCA_011_pre_ax_label_resampled.mha
[slice(110, 142, None), slice(119, 162, None), slice(153, 200, None)]
G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_012\CCF_RectalCA_012_pre_ax_label_raw_resampled.mha
[slice(84, 128, None), slice(94, 144, None), slice(91, 139, None)]
G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_010\CCF_RectalCA_010_pre_ax_label_raw_resampled.mha
[slice(46, 89, None), slice(125, 176, None), slice(93, 131, None)]
G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled\RectalCA_016\CCF_R

In [14]:
missingTumor = []
for name in comList:
    #print("running " +name)
    files = glob.glob(name+"/*.mha")
    if not ("ax" in files[0] and "ax" in files[1]):
        continue
    for file in files:
        if not ("label" in file):
            continue


        image = sitk.ReadImage(file, imageIO="MetaImageIO")
        arr = sitk.GetArrayFromImage(image)
        if not (1 in arr):
            missingTumor.append(file)


print(missingTumor)
        


['G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\\RectalCA_132\\UH_RectalCA_132_pre_ax_raw_label_resampled.mha', 'G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled\\RectalCA_172\\UH_RectalCA_172_pre_ax_raw_label_resampled.mha']
